In [1]:
!pip install datasets transformers nltk --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer


In [3]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/train.csv")
test_df  = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print(train_df.head())


Train shape: (6827, 8)
Test shape: (1707, 2)
   id                                               text  anger  fear  joy  \
0   0  the dentist that did the work apparently did a...      1     0    0   
1   1  i'm gonna absolutely ~~suck~~ be terrible duri...      0     1    0   
2   2  bridge: so leave me drowning calling houston, ...      0     1    0   
3   3  after that mess i went to see my now ex-girlfr...      1     1    0   
4   4  as he stumbled i ran off, afraid it might some...      0     1    0   

   sadness  surprise                    emotions  
0        1         0         ['anger' 'sadness']  
1        1         0          ['fear' 'sadness']  
2        1         0          ['fear' 'sadness']  
3        1         0  ['anger' 'fear' 'sadness']  
4        0         0                    ['fear']  


In [5]:
train_df['clean_text'] = train_df['text'].str.lower()

before_count = train_df['clean_text'].str.len().sum()

translator = str.maketrans('', '', string.punctuation)
train_df['clean_text'] = train_df['clean_text'].apply(lambda x: x.translate(translator))

after_count = train_df['clean_text'].str.len().sum()

reduction_percent = ((before_count - after_count) / before_count) * 100
print(f"Percentage reduction in character count: {reduction_percent:.2f}%")

words = []
for text in train_df['clean_text']:
    words.extend(text.split())

unique_words = set(words)
print("Total unique words:", len(unique_words))

stop_words = set(stopwords.words('english'))
common_stopwords = unique_words.intersection(stop_words)
stopword_percent = (len(common_stopwords) / len(unique_words)) * 100
print(f"Percentage of unique words that are stop words: {stopword_percent:.2f}%")

filtered_words = [w for w in words if w not in stop_words]
word_freq = Counter(filtered_words)
most_common_words = word_freq.most_common(10)

print("\nTop 10 most frequent non-stop words:")
for i, (word, count) in enumerate(most_common_words, 1):
    print(f"{i}. {word} - {count}")

fifth_most_freq_word = most_common_words[4][0]
print(f"\n5th most frequent word (excluding stop words): '{fifth_most_freq_word}'")


Percentage reduction in character count: 3.26%
Total unique words: 8365
Percentage of unique words that are stop words: 1.54%

Top 10 most frequent non-stop words:
1. head - 539
2. eyes - 438
3. like - 394
4. back - 365
5. heart - 334
6. one - 323
7. face - 293
8. get - 291
9. time - 271
10. still - 271

5th most frequent word (excluding stop words): 'heart'


In [6]:
labels = ['anger', 'fear', 'joy', 'sadness', 'surprise']

print("\nInitial rows:", train_df.shape[0])
print("Missing text rows in train:", train_df['text'].isnull().sum())

train_df = train_df.dropna(subset=['text']).reset_index(drop=True)

print("Rows after dropping null-text:", train_df.shape[0])

for l in labels:
    train_df[l] = train_df[l].astype(int)

if 'emotions' in train_df.columns:
    train_df = train_df.drop(columns=['emotions'])

print("\nLabel counts (whole train):")
print(train_df[labels].sum())

n_labels = train_df[labels].sum(axis=1)

print("\nDistribution of #labels per sample (whole train):")
print(n_labels.value_counts().sort_index())



Initial rows: 6827
Missing text rows in train: 0
Rows after dropping null-text: 6827

Label counts (whole train):
anger        808
fear        3860
joy         1660
sadness     2171
surprise    1999
dtype: int64

Distribution of #labels per sample (whole train):
0     676
1    2743
2    2587
3     706
4     112
5       3
Name: count, dtype: int64


In [7]:
train_data, val_data = train_test_split(
    train_df,
    test_size=0.10,
    random_state=42,
    shuffle=True
)

print(f"\nTrain rows: {len(train_data)}    |    Val rows: {len(val_data)}")
print("\nLabel counts (train split):")
print(train_data[labels].sum())
print("\nLabel counts (val split):")
print(val_data[labels].sum())

train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_data.reset_index(drop=True))

print("\nExample training sample (dict):")
print(train_dataset[0])

print("\nTest missing text rows:", test_df['text'].isnull().sum())



Train rows: 6144    |    Val rows: 683

Label counts (train split):
anger        728
fear        3468
joy         1497
sadness     1961
surprise    1798
dtype: int64

Label counts (val split):
anger        80
fear        392
joy         163
sadness     210
surprise    201
dtype: int64

Example training sample (dict):
{'id': 2276, 'text': 'it leaves both my hands free to be able to type.', 'anger': 0, 'fear': 0, 'joy': 1, 'sadness': 0, 'surprise': 0, 'clean_text': 'it leaves both my hands free to be able to type'}

Test missing text rows: 0


In [8]:
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized   = val_dataset.map(tokenize_function, batched=True)

label_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']

for col in label_cols:
    train_tokenized = train_tokenized.rename_column(col, f"labels_{col}")
    val_tokenized   = val_tokenized.rename_column(col,   f"labels_{col}")

train_tokenized.set_format(
    "torch",
    columns=["input_ids", "attention_mask"] + [f"labels_{l}" for l in label_cols]
)

val_tokenized.set_format(
    "torch",
    columns=["input_ids", "attention_mask"] + [f"labels_{l}" for l in label_cols]
)

print("\nTokenization complete!")
print("Train tokenized sample:")
print(train_tokenized[0])


Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/683 [00:00<?, ? examples/s]


✅ Tokenization complete!
Train tokenized sample:
{'labels_anger': tensor(0), 'labels_fear': tensor(0), 'labels_joy': tensor(1), 'labels_sadness': tensor(0), 'labels_surprise': tensor(0), 'input_ids': tensor([   0,  405, 3607,  258,  127, 1420,  481,    7,   28,  441,    7, 1907,
           4,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,